## Install dependencies

In [ ]:
!pip install langchain-community langchain-core pinecone fastembed google-cloud-translate langchain-google-genai langchain-pinecone bert_score

In [ ]:
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import PromptTemplate
from google.colab import files
from bert_score import score

import pandas as pd
import json
from google.colab import userdata

## Implement the Retrieval Chain

In [ ]:
class RetrievalChain:
    def __init__(self, embedding_model, vector_store, top_k=15):
        self.embedding_model = embedding_model
        self.vector_store = vector_store
        self.top_k = top_k

    def get_retrieved_context_ids(self, query):
        """
        Returns ranked list of retrieved context_ids (top_k)
        """
        results = self.vector_store.similarity_search_with_score(
            query,
            k=self.top_k
        )

        return [doc.metadata["context_id"] for doc, _ in results]

    def compute_recall_flags(self, gt_context_id, retrieved_ids):
        """
        Computes Recall@k flags
        """
        return {
            "r@1": int(gt_context_id in retrieved_ids[:1]),
            "r@3": int(gt_context_id in retrieved_ids[:3]),
            "r@5": int(gt_context_id in retrieved_ids[:5]),
            "r@10": int(gt_context_id in retrieved_ids[:10]),
            "r@15": int(gt_context_id in retrieved_ids[:15]),
        }

    def run(self, question_id, query, gt_context_id):
        retrieved_ids = self.get_retrieved_context_ids(query)

        recall_flags = self.compute_recall_flags(
            gt_context_id,
            retrieved_ids
        )

        return {
            "question_id": question_id,
            **recall_flags,
            "contexts": "|".join(map(str, retrieved_ids)),
            "ground_truth": gt_context_id
        }


## Initiaize the RetrievalChain

In [ ]:
embedding_model = FastEmbedEmbeddings()

pc_index_name = "gic-fastembed"
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))
index = pc.Index(pc_index_name)
vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

retrieval_chain = RetrievalChain(
    embedding_model=embedding_model,
    vector_store=vector_store,
)

## Load test data

In [ ]:
import pandas as pd
qa_data = pd.read_csv("gic_qa_with_ids.csv")
qa_data.head(1)

## tamil questions

In [ ]:
results_ta = []

for _, row in qa_data.iterrows():
    print(f"Processing question: {row['question_id']}")
    # print(f"Question: {row['question_ta']}")
    result = retrieval_chain.run(
        question_id=row["question_id"],
        query=row["question_ta"],
        gt_context_id=row["context_id"]
    )
    # print("Ground truth: ", row["context_id"])
    # print("Retrieved: ", result["contexts"])
    results_ta.append(result)

In [ ]:
df = pd.DataFrame(results_ta)
df.tail(5)

In [ ]:
df.to_csv(f"result_ta_baseline.csv", index=False)

In [ ]:
summary_baseline_ta = {
    "Recall@1": df["r@1"].mean(),
    "Recall@3": df["r@3"].mean(),
    "Recall@5": df["r@5"].mean(),
    "Recall@10": df["r@10"].mean(),
    "Recall@15": df["r@15"].mean(),
}

summary_baseline_ta

In [ ]:
summary_pct_baseline_ta = {
    k: v * 100
    for k, v in summary_baseline_ta.items()
}

summary_pct_baseline_ta

In [ ]:
summary_pct_baseline_ta['language'] = "ta"
summary_pct_baseline_ta

## Sinhala questions

In [ ]:
results_si = []

for _, row in qa_data.iterrows():
    print(f"Processing question: {row['question_id']}")
    # print(f"Question: {row['question_ta']}")
    result = retrieval_chain.run(
        question_id=row["question_id"],
        query=row["question_si"],
        gt_context_id=row["context_id"]
    )
    # print("Ground truth: ", row["context_id"])
    # print("Retrieved: ", result["contexts"])
    results_si.append(result)

Processing question: 1
Processing question: 2
Processing question: 3
Processing question: 4
Processing question: 5
Processing question: 6
Processing question: 7
Processing question: 8
Processing question: 9
Processing question: 10
Processing question: 11
Processing question: 12
Processing question: 13
Processing question: 14
Processing question: 15
Processing question: 16
Processing question: 17
Processing question: 18
Processing question: 19
Processing question: 20
Processing question: 21
Processing question: 22
Processing question: 23
Processing question: 24
Processing question: 25
Processing question: 26
Processing question: 27
Processing question: 28
Processing question: 29
Processing question: 30
Processing question: 31
Processing question: 32
Processing question: 33
Processing question: 34
Processing question: 35
Processing question: 36
Processing question: 37
Processing question: 38
Processing question: 39
Processing question: 40
Processing question: 41
Processing question: 42
P

In [ ]:
df = pd.DataFrame(results_si)
df.tail(5)

In [ ]:
df.to_csv(f"result_si_baseline.csv", index=False)

In [ ]:
summary_baseline_si = {
    "Recall@1": df["r@1"].mean(),
    "Recall@3": df["r@3"].mean(),
    "Recall@5": df["r@5"].mean(),
    "Recall@10": df["r@10"].mean(),
    "Recall@15": df["r@15"].mean(),
}

summary_baseline_si

In [ ]:
summary_pct_baseline_si = {
    k: v * 100
    for k, v in summary_baseline_si.items()
}

summary_pct_baseline_si

In [ ]:
summary_pct_baseline_si['language'] = "si"
summary_pct_baseline_si

## Overall summary

In [ ]:
summary_baseline = pd.DataFrame([summary_pct_baseline_ta, summary_pct_baseline_si])
summary_baseline

,Recall@1,Recall@3,Recall@5,Recall@10,Recall@15,language
0,0.6,1.8,2.0,3.8,4.2,ta
1,1.6,3.8,5.6,7.4,8.2,si


In [ ]:
cols = ["language"] + [c for c in summary_baseline.columns if c != "language"]
summary_baseline = summary_baseline[cols]
summary_baseline

,language,Recall@1,Recall@3,Recall@5,Recall@10,Recall@15
0,ta,0.6,1.8,2.0,3.8,4.2
1,si,1.6,3.8,5.6,7.4,8.2


In [ ]:
summary_baseline.to_csv(f"summary_baseline.csv", index=False)